## Выбор модели для задач классификации

Для задач классификации существует несколько способов выбора модели:

1. **Обучить маленькую модель с нуля**: Выбор легковесной архитектуры модели и полное обучение её на нашем наборе данных.
2. **Fine-Tuning предобученной модели**: Используем модель, которая была предобучена авторами на другом наборе данных, и только последние слои переобучается на нашем конкретном наборе данных.
3. **Использовать предобученные веса напрямую**: В этом методе используется предобученная модель без дополнительного обучения.

Лучшим вариантом является Fine-Tuning, поэтому в качестве бейзлайна возьмем маленькую (5.3M) EfficientNet B0 с претрейновыми параметрами

[Оригинальная статья](https://arxiv.org/abs/1905.11946)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from PIL import Image

In [2]:
import os
import zipfile
import json

In [3]:
!pip freeze > requirements.txt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Загрузка гиперпараметров из JSON (создайте папку classification, если её еще нет в корне диска Google Drive)
with open('/content/drive/MyDrive/classification/hyperparams.json', "r") as f:
  hyperparams = json.load(f)


**Структура** c*lassification_dataset.zip*: (внутри папка с таким же названием)

```
classification_dataset/
├── train/
│   ├── 0/
│   └── 1/
└── test/
    ├── 0/
    └── 1/
```



In [6]:
local_zip = '/content/drive/MyDrive/classification/classification_dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/dataset')
zip_ref.close()

In [10]:
# Класс для аугментации данных
class AugmentedDataset(Dataset):
    def __init__(self, original_folder, target_size=50, transform=None):
        self.transform = transform
        self.samples = []
        self.class_to_idx = {}

        class_names = sorted(os.listdir(original_folder)) # Получаем отсортированный список имен классов
        for idx, class_name in enumerate(class_names): # Итерируемся и присваиваем индексы
            self.class_to_idx[class_name] = idx

        # Собираем пути к изображениям для каждого класса
        for class_name in class_names: # Используем отсортированные имена классов
            class_path = os.path.join(original_folder, class_name)
            images = [os.path.join(class_path, img) for img in os.listdir(class_path)]
            # Повторяем изображения до достижения целевого размера
            for i in range(target_size):
                self.samples.append((images[i % len(images)], class_name))

    # Возвращает общее количество элементов в наборе данных
    def __len__(self):
        return len(self.samples)

    # Возвращает одно изображение и его метку по индексу
    def __getitem__(self, idx):
        img_path, class_name = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        # Применяем аугментации
        if self.transform:
            image = self.transform(image)

        # Преобразуем метку класса в числовой формат
        label_str = class_name # Метка класса все еще строка
        label = self.class_to_idx[label_str] # Используем class_to_idx для получения числового индекса
        return image, torch.tensor(label, dtype=torch.float32)

In [11]:
# Трансформы с аугментациями для тренировочных данных
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Трансформы для тестовых данных
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Создание аугментированных датасетов
train_dataset = AugmentedDataset(
    original_folder='/content/dataset/classification_dataset/train',
    target_size=50,
    transform=train_transform
)

test_dataset = datasets.ImageFolder(
    '/content/dataset/classification_dataset/test',
    transform=test_transform
)

# DataLoader'ы
train_loader = DataLoader(
    train_dataset,
    batch_size=hyperparams['batch_size'],
    shuffle=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=hyperparams['batch_size'],
    shuffle=False
)

In [12]:
# Проверим классы
print(train_dataset.class_to_idx)

{'0': 0, '1': 1}


In [ ]:
# Загрузка модели EfficientNet
model = efficientnet_b0(EfficientNet_B0_Weights.DEFAULT)

# Заморозка всех слоев, кроме последнего
for param in model.parameters():
    param.requires_grad = False

# Замена финального классификатора
num_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Linear(128, 1),  # Бинарная классификация
    nn.Sigmoid()        # Для вероятностей
)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 43.1MB/s]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.BCELoss()  # Для бинарной классификации
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=hyperparams['learning_rate'],
    weight_decay=hyperparams['weight_decay']
)

In [ ]:
# Обучение модели
for epoch in range(hyperparams['num_epochs']):
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float()

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    # Валидация
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float()

            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    # Вывод статистики
    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(test_loader.dataset)
    accuracy = correct / total

    print(f"Epoch {epoch+1}/{hyperparams['num_epochs']}")
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}")

Epoch 1/10
Train Loss: 0.6868, Val Loss: 0.6652, Accuracy: 0.7900
Epoch 2/10
Train Loss: 0.6343, Val Loss: 0.6405, Accuracy: 0.8680
Epoch 3/10
Train Loss: 0.6046, Val Loss: 0.6199, Accuracy: 0.8740
Epoch 4/10
Train Loss: 0.5463, Val Loss: 0.5986, Accuracy: 0.8800
Epoch 5/10
Train Loss: 0.5169, Val Loss: 0.5565, Accuracy: 0.8950
Epoch 6/10
Train Loss: 0.4488, Val Loss: 0.5336, Accuracy: 0.9010
Epoch 7/10
Train Loss: 0.4102, Val Loss: 0.5096, Accuracy: 0.8950
Epoch 8/10
Train Loss: 0.3712, Val Loss: 0.4839, Accuracy: 0.8970
Epoch 9/10
Train Loss: 0.3392, Val Loss: 0.4623, Accuracy: 0.8910
Epoch 10/10
Train Loss: 0.3108, Val Loss: 0.4429, Accuracy: 0.8940


In [ ]:
# Сохранение модели
torch.save(model, '/content/trained_model_classification.pt')

# Копирование на Google Drive
!cp "/content/trained_model_classification.pt" "/content/drive/MyDrive/classification/trained_model_classification.pt"
print("Модель сохранена")

Модель сохранена
